### To Do
- ~~Add a hash to every ActionObject~~
- ~~Make every ActionObject that's created from other ActionObjects have an automatic hash ~~

- Bonus: 
    - Maybe see if we can stitch together Code Verification and Code Submission- enable the "Trace" feature!
    - Check if ActionObjects work with the dataset API

### Scenario

a = ActionObject(data1)\
a.history_hash = None\
b = ActionObject(data2)\
b.history_hash = None\
c = a + b\
c.history_hash = hash(a, b, add)\
d = a + b\
d.history_hash = hash(a, b, add)\
c.history_hash == d.history_hash\

e = a * b\
e.history_hash = hash(a, b, mul)

f = ActionObject(data1)
g = f + b



### Takeaways:
- Different ops with the same ActionObjects should have different hashes
(e and c should not have the same hash history)
- The same ActionObjects and op should always produce the same hash
(c and d should have the same hash history)
- Different ActionObjects with the same value should still have different hashes. (Check with Madhava)
(c and g should have different hash history because f might refer to the same data but in a different dataset/asset)
- Be able to distinguish between std() and std(axis=1)
- Be able to distinguish between a + 5 and a + 6 even if 5 and 6 aren't ActionObjects
    - what if we're adding a + np.array([1]) and a + np.array([2]) -> since np.arrays aren't hashable? 

### Troubles

- Will we run into trouble with np.add vs torch.add vs regular add? \shrug

Do we really need hashes for this to work? Could we use f"{parent ID}_{OPname}"
- Could get gigantic strings if we're adding a billion things?



In [1]:
from typing import Hashable

In [2]:
from syft.core.node.new.action_object import ActionObject

In [3]:
import numpy as np

In [4]:
data = 2*np.random.rand(10,10)-1

In [5]:
a = ActionObject(syft_action_data=data)
a.id

Lights... Cameras... ACTION OBJECT!


<UID: 47434b44697d4c59a3c3b87678e66cee>

In [6]:
a.std(1)

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 466454064648700083

In [7]:
a.std(axis=1)

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 466454064648700083

In [8]:
a.std(axis=0)

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: -6588866534066588974

In [6]:
b = ActionObject(syft_action_data=data)
b.id

Lights... Cameras... ACTION OBJECT!


<UID: b72ffc4777e54d158f81002355ab9e25>

In [9]:
a + 6

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: -3711062134419802598

In [10]:
a + 5

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: -4838454943041321254

In [11]:
p = a + 5

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


In [12]:
v = a.transpose()

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


In [13]:
v = a + a
v

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 6131213075072729900

In [14]:
v = a == a
v

We're making history... hashes.
Lights... Cameras... ACTION OBJECT!


History: 9153372762820366171

In [14]:
v

History: 8993198827105615052

In [15]:
type(v.id)

syft.core.common.uid.UID

In [16]:
v2 = a + 6
v2

We're making history
Lights... Cameras... ACTION OBJECT!


History: -18576999663428702

In [17]:
a + 5

We're making history
Lights... Cameras... ACTION OBJECT!


History: -4829506950524717899

In [18]:
a.id, b.id

(<UID: 1fb24680501d48fe82da4e6d81066c3d>,
 <UID: b72ffc4777e54d158f81002355ab9e25>)

In [19]:
c = a + b
c

We're making history
Lights... Cameras... ACTION OBJECT!


History: 838615051158369992

In [20]:
a + b

We're making history
Lights... Cameras... ACTION OBJECT!


History: 838615051158369992

In [21]:
c.syft_parent_id

[<UID: b72ffc4777e54d158f81002355ab9e25>]

In [22]:
d = b + a
d

We're making history
Lights... Cameras... ACTION OBJECT!


History: -7447429398961264511

In [21]:
e = a + b
e

<class 'tuple'> (<ufunc 'add'>, (array([[ 0.26320744,  0.42057623,  0.97821957,  0.66036263, -0.455208  ,
         0.53522391,  0.31649407,  0.0407967 ,  0.42753522,  0.96334865],
       [ 0.05144174,  0.20800653, -0.41269607, -0.51113985,  0.18452361,
        -0.13980118, -0.40930577, -0.37202163,  0.25566948, -0.46311769],
       [-0.32654568, -0.27502359,  0.07283238,  0.38464479,  0.23018911,
        -0.36986018,  0.47538614, -0.35619602, -0.95752165,  0.59723821],
       [-0.19106782, -0.0781314 ,  0.94524308,  0.69524781, -0.23514045,
        -0.55967342, -0.2552866 ,  0.91086411,  0.24642226, -0.50820123],
       [ 0.51503809, -0.26179417,  0.68411117,  0.75031138, -0.20807713,
         0.23333458, -0.40221741,  0.30878203,  0.74170343,  0.8424877 ],
       [-0.59766815, -0.02380914,  0.21513482, -0.70394137,  0.50588239,
        -0.98755633, -0.51765294, -0.96239706, -0.47455299, -0.88733002],
       [ 0.58632083,  0.3242106 , -0.76415867,  0.85953175,  0.80015576,
         0.4

History: 1587900562259335929

In [22]:
hash

<function hash(obj, /)>

In [23]:
hash(a.id)

351961006087292445

In [24]:
type(hash(a.id))

int

In [25]:
a == a

<class 'tuple'> (<ufunc 'equal'>, (array([[ 0.26320744,  0.42057623,  0.97821957,  0.66036263, -0.455208  ,
         0.53522391,  0.31649407,  0.0407967 ,  0.42753522,  0.96334865],
       [ 0.05144174,  0.20800653, -0.41269607, -0.51113985,  0.18452361,
        -0.13980118, -0.40930577, -0.37202163,  0.25566948, -0.46311769],
       [-0.32654568, -0.27502359,  0.07283238,  0.38464479,  0.23018911,
        -0.36986018,  0.47538614, -0.35619602, -0.95752165,  0.59723821],
       [-0.19106782, -0.0781314 ,  0.94524308,  0.69524781, -0.23514045,
        -0.55967342, -0.2552866 ,  0.91086411,  0.24642226, -0.50820123],
       [ 0.51503809, -0.26179417,  0.68411117,  0.75031138, -0.20807713,
         0.23333458, -0.40221741,  0.30878203,  0.74170343,  0.8424877 ],
       [-0.59766815, -0.02380914,  0.21513482, -0.70394137,  0.50588239,
        -0.98755633, -0.51765294, -0.96239706, -0.47455299, -0.88733002],
       [ 0.58632083,  0.3242106 , -0.76415867,  0.85953175,  0.80015576,
         0

History: -8278011067452327721

In [26]:
c = a + 5

<class 'int'> 5
1 [<UID: cafdfa6762b84c4bacf2977080dce38d>]
[<UID: cafdfa6762b84c4bacf2977080dce38d>]
We got a parent and their op!
-7044735904885987341  <--- this is my history hash
Lights... Cameras... ACTION OBJECT!


In [27]:
c.syft_parent_id

<UID: cafdfa6762b84c4bacf2977080dce38d>

In [28]:
a.std()

1 [<UID: cafdfa6762b84c4bacf2977080dce38d>]
[<UID: cafdfa6762b84c4bacf2977080dce38d>]
We got a parent and their op!
-7794151430135992769  <--- this is my history hash
Lights... Cameras... ACTION OBJECT!


History: -7794151430135992769

In [29]:
type(a==a)

<class 'tuple'> (<ufunc 'equal'>, (array([[ 0.26320744,  0.42057623,  0.97821957,  0.66036263, -0.455208  ,
         0.53522391,  0.31649407,  0.0407967 ,  0.42753522,  0.96334865],
       [ 0.05144174,  0.20800653, -0.41269607, -0.51113985,  0.18452361,
        -0.13980118, -0.40930577, -0.37202163,  0.25566948, -0.46311769],
       [-0.32654568, -0.27502359,  0.07283238,  0.38464479,  0.23018911,
        -0.36986018,  0.47538614, -0.35619602, -0.95752165,  0.59723821],
       [-0.19106782, -0.0781314 ,  0.94524308,  0.69524781, -0.23514045,
        -0.55967342, -0.2552866 ,  0.91086411,  0.24642226, -0.50820123],
       [ 0.51503809, -0.26179417,  0.68411117,  0.75031138, -0.20807713,
         0.23333458, -0.40221741,  0.30878203,  0.74170343,  0.8424877 ],
       [-0.59766815, -0.02380914,  0.21513482, -0.70394137,  0.50588239,
        -0.98755633, -0.51765294, -0.96239706, -0.47455299, -0.88733002],
       [ 0.58632083,  0.3242106 , -0.76415867,  0.85953175,  0.80015576,
         0

syft.core.node.new.action_object.ActionObject

In [30]:
b = a + a

<class 'tuple'> (<ufunc 'add'>, (array([[ 0.26320744,  0.42057623,  0.97821957,  0.66036263, -0.455208  ,
         0.53522391,  0.31649407,  0.0407967 ,  0.42753522,  0.96334865],
       [ 0.05144174,  0.20800653, -0.41269607, -0.51113985,  0.18452361,
        -0.13980118, -0.40930577, -0.37202163,  0.25566948, -0.46311769],
       [-0.32654568, -0.27502359,  0.07283238,  0.38464479,  0.23018911,
        -0.36986018,  0.47538614, -0.35619602, -0.95752165,  0.59723821],
       [-0.19106782, -0.0781314 ,  0.94524308,  0.69524781, -0.23514045,
        -0.55967342, -0.2552866 ,  0.91086411,  0.24642226, -0.50820123],
       [ 0.51503809, -0.26179417,  0.68411117,  0.75031138, -0.20807713,
         0.23333458, -0.40221741,  0.30878203,  0.74170343,  0.8424877 ],
       [-0.59766815, -0.02380914,  0.21513482, -0.70394137,  0.50588239,
        -0.98755633, -0.51765294, -0.96239706, -0.47455299, -0.88733002],
       [ 0.58632083,  0.3242106 , -0.76415867,  0.85953175,  0.80015576,
         0.4

In [31]:
b.syft_parent_id

<UID: cafdfa6762b84c4bacf2977080dce38d>

In [ ]:
from datetime import datetime

In [ ]:
datetime.now()

In [ ]:
from typing import Callable
import numpy as np

# This is our decorator
def submit_to_domain_node(user_func: Callable, user_input: np.ndarray, policy, constraint, output_args):
    print("We would try to check that the verify key is correct")
    print("We 

In [ ]:
def parent(num):
    
    def first_child():
        return "Emily"
    
    def second_child():
        return "George"
    
    if num == 1:
        return first_child
    else:
        return second_child

In [45]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print("Something happens before the function")
        print(args)
        print(kwargs)
        for k in kwargs:
            print(k, type(k))
        print([hash(kwarg) for kwarg in kwargs])
        func()
        print("Something happens after the function")
    return wrapper

In [46]:
def say_whee(num, setup):
    print(setup)
    print("Whee!" * num)

say_whee = decorator(say_whee)

In [47]:
say_whee(5, setup=10)

Something happens before the function
(5,)
{'setup': 10}
setup <class 'str'>
[-8415799926645800496]


TypeError: say_whee() missing 2 required positional arguments: 'num' and 'setup'

In [ ]:
@decorator
def say_boo():
    print("Booo!")

In [ ]:
say_boo()